In [ ]:
import subprocess
from asyncio import Queue, gather, run, create_task
from glob import glob
from os.path import join, basename

In [ ]:
WORKS = 10

async def log(url):
    with open('log.csv', 'a') as f:
        print(f'Error na url: {url}')
        f.write(f'{datetime.now()};{url}\n')
        
        
async def get_path_mp4(path_mp4):
    plays = Queue()
    mp4s = glob(join(path_mp4, '*.mp4'))
    for e, path in enumerate(mp4s):
        await plays.put((path, e))
    return plays


async def mp4_to_mp3(input_file, output_file):
    print(basename(input_file), output_file)
    try:
        p = subprocess.Popen(['ffmpeg', '-i', input_file, '-vn', '-acodec', 'libmp3lame', '-y', output_file])
        await p.communicate()
    except:
        log(output_file)


async def convert(queue, path_output):
    while queue.qsize() != 0:
        input_path = await queue.get()
        await mp4_to_mp3(input_path[0], join(path_output, basename(input_path[0]))[:-1] + '3')


async def main():
    path_input = '/home/gregorio/Documentos/PROJETOS/ESTUDOS/SCRIPTS/'
    path_output = '/home/gregorio/Documentos/PROJETOS/ESTUDOS/SCRIPTS/mp3'
    mp4s_list = await get_path_mp4(path_input)
    print(f'lenght list: {mp4s_list.qsize()}')

    tasks = []
    for _ in range(WORKS):
        tasks.append(create_task(convert(mp4s_list, path_output)))
    return await gather(*tasks)

In [ ]:
run(main())